# Project 1
## Machine Learning without Mllib Pipeline

%pip install 

In [8]:
import findspark
from pyspark.sql import SparkSession

In [9]:
findspark.init()
findspark.find()

'c:\\Users\\ellin\\Documents\\MSc_IoT_MAU\\DA642E-AI-and-Data-Management-for-IOT\\.venv\\Lib\\site-packages\\pyspark'

In [10]:
spark = SparkSession \
    .builder \
    .appName ("Titanic Data") \
    .getOrCreate()

In [11]:
spark

In [12]:
df = (spark.read
        .format("csv")
        .option("header","true")
        .load("data/train.csv")
)

In [13]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [14]:
from pyspark.sql.functions import col

In [15]:
dataset = df.select(col('Survived').cast('float'),
                    col('Pclass').cast('float'),
                    col('Sex'),
                    col('Age').cast('float'),
                    col('Fare').cast('float'),
                    col('Embarked')
                    )

In [16]:
dataset.show(4)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
+--------+------+------+----+-------+--------+
only showing top 4 rows



In [17]:
from pyspark.sql.functions import isnull, when, count, col

In [18]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [19]:
dataset = dataset.replace('?', None)\
            .dropna(how='any')

In [20]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [21]:
import numpy as np
from pyspark.ml.feature import StringIndexer

In [22]:
dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [23]:
dataset = StringIndexer(
    inputCol='Embarked',
    outputCol='Boarded',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [24]:
dataset.show(2)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 2 rows



In [25]:
# Drop unnecessary columns
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

In [26]:
# Assemble features with VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [27]:
require_featured = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
assembler = VectorAssembler(inputCols=require_featured, outputCol='features')
transformed_data = assembler.transform(dataset)

In [28]:
transformed_data.show(5)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 5 rows



In [29]:
# split train adn test
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print("N training samples: " + str(training_data.count()))
print("N testing samples: " + str(test_data.count()))

N training samples: 578
N testing samples: 134


In [30]:
from pyspark.ml.classification import RandomForestClassifier

In [31]:
rf = RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)

In [32]:
model = rf.fit(training_data)

In [33]:
predictions = model.transform(test_data)

In [34]:
# Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived',
    predictionCol='prediction',
    metricName='accuracy')

In [36]:
accuracy = evaluator.evaluate(predictions)
print('Training Accuracy= ', accuracy)

Training Accuracy=  0.8208955223880597


# Project 2
## Machine Learning Project with Mllib Pipeline

In [37]:
# setup pyspark env

# download JVM
## %pip apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [38]:
# libraries

from pyspark.sql import SparkSession
import findspark

In [39]:
# Cell for debugging 
findspark.init()
findspark.find()

'c:\\Users\\ellin\\Documents\\MSc_IoT_MAU\\DA642E-AI-and-Data-Management-for-IOT\\.venv\\Lib\\site-packages\\pyspark'

set path_to_spark_archive to sparks PATH
path_to_spark_archive = 'c:/users/ellin/appdata/local/programs/python/python311/lib/site-packages'

#!tar xf "{path_tospark_archive}"

import tarfile
with tarfile.open(path_to_spark_archive, "r:*") as tar:
    tar.extractall(path=path_to_spark_archive)

In [40]:
spark = SparkSession.builder \
    .master("local") \
    .appName ("Titanic Data") \
    .getOrCreate()

spark

In [41]:
df = (spark.read
        .format("csv")
        .option("header", "true")
        .load("data/train.csv"))

df.show(3)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [42]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import RandomForestClassifier

In [43]:
from pyspark.ml import Pipeline

In [46]:
(train_df, test_df) = dataset.randomSplit([0.8, 0.2], 11)
print("N of train samples: " + str(train_df.count()))
print("N of test samples: " + str(test_df.count()))

N of train samples: 562
N of test samples: 150


In [47]:
Sex_indexer = StringIndexer(inputCol="Sex", outputCol="Gender")
Embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Boarded")

inputCols = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
outputCol="features"

vector_assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)

dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")


In [48]:
pipeline = Pipeline(stages=[Sex_indexer, Embarked_indexer, vector_assembler, dt_model])

final_pipeline = pipeline.fit(train_df)

test_predictions_pipeline = final_pipeline.transform(test_df)

test_predictions_pipeline.show(5, truncate=False)

Py4JJavaError: An error occurred while calling o429.fit.
: org.apache.spark.SparkException: Input column Sex does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.